# 제주도 구간별 일일 평균 통행속도 데이터 -> 월평균 통행속도 table 생성

In [5]:
import requests
import json
import re
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

In [ ]:
df=pd.read_parquet('./dataset/제주일별속도.parquet')
tqdm.pandas()
df['month']=df.progress_apply(lambda row : int(row['base_date_hour'][4:6]),axis=1)
result=df.groupby(['month']).mean()
result['month']=result.index
result[['month','speed']].to_parquet('./dataset/제주월별평속2019.parquet',index=False)

# 2019 월별 속도 Column 추가

In [92]:
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('.dataset/test.csv')
mspeed_19 = pd.read_parquet('./dataset/제주월별평속2019.parquet')

In [95]:
tqdm.pandas()
train['month']= train.progress_apply(lambda x : int(str(x['base_date'])[4:6]),axis=1)
test['month']= test.progress_apply(lambda x : int(str(x['base_date'])[4:6]),axis=1)

100%|██████████| 291241/291241 [00:05<00:00, 51538.58it/s]


In [ ]:
train=pd.merge(train,mspeed_19,how='left', left_on=['month'],right_on=['month']).drop(columns=['month']).rename(columns={'speed':'month_mean'})
test=pd.merge(test,mspeed_19,how='left', left_on=['month'],right_on=['month']).drop(columns=['month']).rename(columns={'speed':'month_mean'})

In [91]:
train.to_parquet('./dataset/train_파생변수.parquet',index=False)
test.to_parquet('./dataset/test_파생변수.parquet',index=False)